In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vision-transformer/tensorflow2/vit-b16-classification/1/saved_model.pb
/kaggle/input/vision-transformer/tensorflow2/vit-b16-classification/1/variables/variables.index
/kaggle/input/vision-transformer/tensorflow2/vit-b16-classification/1/variables/variables.data-00000-of-00001


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models
import tf_keras 

2025-10-21 12:05:12.870221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761048312.893773     201 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761048312.901182     201 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
num_classes = 10

In [4]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")


In [5]:
def preprocess(images, labels):
    images = tf.image.resize(images, (224, 224))
    images = (images / 127.5) - 1.0   # scale to [-1,1]
    return images, tf.squeeze(labels)

In [6]:
train_ds = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(50000)
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

I0000 00:00:1761048320.082374     201 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1761048320.083030     201 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [7]:
vit_url = "https://www.kaggle.com/models/spsayakpaul/vision-transformer/TensorFlow2/vit-b16-classification/1"

base_model = hub.KerasLayer(vit_url, trainable=True, input_shape=(224, 224, 3))


In [8]:

model = tf_keras.Sequential([
    tf_keras.layers.InputLayer(input_shape=(224, 224, 3)),
    base_model,
    tf_keras.layers.Dropout(rate=0.2),
    tf_keras.layers.Dense(num_classes, activation="softmax")
])

model.build((None,) + (224, 224, 3))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              86567656  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 10)                10010     
                                                                 
Total params: 86577666 (330.27 MB)
Trainable params: 10010 (39.10 KB)
Non-trainable params: 86567656 (330.23 MB)
_________________________________________________________________


In [9]:
model.compile(
    optimizer=tf_keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(
    monitor='val_loss',   
    patience=7,           
    restore_best_weights=True  
)

checkpoint = ModelCheckpoint(
    'best_model.h5',      
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5,
    callbacks=[early_stop, checkpoint]
)


Epoch 1/50


I0000 00:00:1761048331.540740     237 service.cc:148] XLA service 0x7fd7c8006400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761048331.540802     237 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1761048331.540808     237 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1761048332.273132     237 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1761048334.684411     237 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 [==============================] - 842s 535ms/step - loss: 2.2856 - accuracy: 0.3898 - val_loss: 0.6949 - val_accuracy: 0.7747
Epoch 2/50
1563/1563 [==============================] - 831s 532ms/step - loss: 0.7317 - accuracy: 0.7566 - val_loss: 0.3220 - val_accuracy: 0.9025
Epoch 3/50
1563/1563 [==============================] - 831s 532ms/step - loss: 0.4233 - accuracy: 0.8641 - val_loss: 0.2174 - val_accuracy: 0.9317
Epoch 4/50
1563/1563 [==============================] - 831s 532ms/step - loss: 0.3086 - accuracy: 0.9036 - val_loss: 0.1749 - val_accuracy: 0.9456
Epoch 5/50
1563/1563 [==============================] - 831s 532ms/step - loss: 0.2518 - accuracy: 0.9199 - val_loss: 0.1529 - val_accuracy: 0.9532
Epoch 6/50
1563/1563 [==============================] - 831s 532ms/step - loss: 0.2175 - accuracy: 0.9324 - val_loss: 0.1395 - val_accuracy: 0.9567
Epoch 7/50
1553/1563 [============================>.] - ETA: 4s - loss: 0.1972 - accuracy: 0.9391

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc * 100:.2f}%")